In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('transactions_dataset.csv', sep = ";")

df.head()

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051


In [4]:
from itertools import combinations
from collections import Counter
import pandas as pd

# Limit the df to the last 2 million rows
if len(df) > 2000000:
    df = df.tail(2000000)

# Convert date_order to datetime format and find the latest date
df['date_order'] = pd.to_datetime(df['date_order'])
latest_date = df['date_order'].max()

# Determine the start date for the last year period
start_date_last_year = latest_date - pd.DateOffset(years=1)

# Filter the data for the last year
data_last_year = df[df['date_order'] > start_date_last_year]

# Group data by client_id and collect all product_ids bought together in the last year
transactions_last_year = data_last_year.groupby('client_id')['product_id'].apply(list)

In [9]:

# Group by 'client_id' and 'date_order' to create a list of products for each customer per day
daily_purchases = data_last_year.groupby(['client_id', 'date_order'])['product_id'].apply(list).reset_index()

# Generate all combinations of product pairs for each daily purchase and count the occurrences of each pair
pair_counts = Counter()
for products in daily_purchases['product_id']:
    for pair in combinations(set(products), 2):
        pair_counts[pair] += 1

# Calculate the individual product frequencies within the data
product_counts = data_last_year['product_id'].value_counts().to_dict()

# Total number of transactions is the number of days where at least one purchase was made by each customer
total_daily_transactions = daily_purchases.shape[0]

# Function to calculate lift for each pair
def calculate_lift(pair, freq, product_counts, total_transactions):
    product_a, product_b = pair
    freq_a = product_counts.get(product_a, 0)
    freq_b = product_counts.get(product_b, 0)
    lift = (freq / total_transactions) / ((freq_a / total_transactions) * (freq_b / total_transactions))
    return lift

# Apply a minimum frequency threshold to the product pairs
# Let's consider products that have a frequency of at least 5
filtered_product_counts = {product: freq for product, freq in product_counts.items() if freq >= 5}
filtered_pair_counts = Counter({pair: count for pair, count in pair_counts.items()
                                if filtered_product_counts.get(pair[0], 0) >= 5 and
                                filtered_product_counts.get(pair[1], 0) >= 5})

# Convert filtered_pair_counts to a DataFrame
filtered_pairs_df = pd.DataFrame(filtered_pair_counts.items(), columns=['Product_Pair', 'Frequency'])

# Recalculate lift for the filtered product pairs
filtered_pairs_df['Lift'] = filtered_pairs_df.apply(
    lambda row: calculate_lift(row['Product_Pair'], row['Frequency'], filtered_product_counts, total_daily_transactions), axis=1
)

# Sort pairs by lift in descending order and select the top 10
top_10_filtered_pairs_lift = filtered_pairs_df.sort_values(by='Lift', ascending=False).head(10)

top_10_filtered_pairs_lift



,Product_Pair,Frequency,Lift
2401317,"(612247, 2948413)",5,65213.4
2401302,"(2681508, 2948413)",5,65213.4
2401301,"(2681508, 612247)",5,65213.4
1043970,"(536874, 2383666)",5,65213.4
3454032,"(2166544, 497232)",5,54344.5
664459,"(1323742, 1235948)",5,54344.5
664458,"(1323742, 893418)",5,54344.5
3482874,"(100874, 968365)",5,54344.5
844855,"(1860609, 2322619)",5,54344.5
554379,"(2375657, 2625022)",5,54344.5


In [8]:
top_10_pairs_lift_last_year

,Product_Pair,Frequency,Lift
4761597,"(2636447, 2923360)",1,326067.0
8753580,"(299568, 1619093)",1,326067.0
3804118,"(728707, 3217132)",1,326067.0
4783163,"(1749888, 204518)",1,326067.0
4783167,"(1749888, 1200363)",1,326067.0
3257110,"(2938139, 2978230)",1,326067.0
3257109,"(2938139, 764885)",1,326067.0
4783169,"(1749888, 2656371)",1,326067.0
2962643,"(659905, 519909)",1,326067.0
4783170,"(1749888, 2907382)",1,326067.0


In [10]:
filtered_pairs_df

,Product_Pair,Frequency,Lift
0,"(2151232, 2885889)",3,0.309926
1,"(2151232, 1468420)",1,0.087998
2,"(2151232, 1639111)",22,40.413027
3,"(2151232, 1108489)",2,0.327480
4,"(2151232, 509131)",5,0.569599
...,...,...,...
7613055,"(2628068, 1731212)",1,2.961230
7613056,"(2628068, 2799312)",1,0.820716
7613057,"(2628068, 683924)",1,0.431077
7613058,"(1298661, 1808264)",1,2.047902
